# Project Title
**Author**: Todd Strain



## Overview
Virtucon have decided to diversify and explore new reevnue strams by offering flight operation services
for commercial and private enterprises. The company is concerned about the risks and environmental impact
of this new line of business. Since the company has little knowledge of these risks we've undertaken an
analysis of these issues. Three datasets were used to determin which models of aircraft are the safest to
operate while having an acceptable level of environmental impact. The result was the Canadaair RJ190 was the
safetst aircraft to operate. The <blank> was the most enironmentaly friendly aircraft. Our reecommendation is 


## Business Problem
Virtucon wants to diversify it's business and introduce new revenu streams by operating aircraft for commercial 
and private enterprises. They want to know the safest and aircraft to operate with the least environmenatl impact.
To answer that question we used the Bureau of Transportation Statistics (BTS) formula:

*“Rates are computed by dividing the number of Fatalities, Seriously injured persons, Total accidents, and Fatal accidents by the number of Aircraft-miles, Aircraft departures, or Flight hours.” *

To determine environmntal impact we used data for popular aircraft from EASA.

## Data Understanding
For aircraft safty we used two datasets NTSB aircraft crash database and BTS T-100 Domestic Segmaent database for the 10 years preceeding 2024.

The NTSB crash database includes aviation accident data from 1962 to 2023 about civil aviation accidents and selected incidents in the United States and international waters. We are most interested in the number of accidents per aircraft model type, and the number and grade of passenger injuries.

The BTS T-100 Domestic Segment database includes on-flight origin and destination records. From this dataset we use the make and model of aircraft, and the number of flight instances. 

In both datasets we've selected data from 2014-2024 (most recently reported.) 


***

### Data Structures
- **df**            DataFrame. Full AviationData.csv provided by project. Containf flight crash data.
- **bts_df**        DataFrame. 5 years of flight data from BTS. Contains airline flight data.
- **ac_df**         DataFrame. Legend that matches aircraft code in bts_df to Make/Model.
- **joined_df**     DataFrame. Merged bts_df with Make/Model from ac_df.
- **bts_actype**    DataFrame. List of aircraft from joined_df. Used to create spreadsheet for standarized names.
- **bts_map_df**    DataFrame. Imported from bts_actype spreadsheet with actype converted to standardized names.
- **bts_map_dic**   Dict. bts_map_df converted to dictionary so it can be added as new column to joined_df.
- **crash_actype**  DataFrame. Copy of Make, Model columns from df. Later updated with standardized model name.
- **custom_ac**     DataFrame. Copy of Model column from crash_actype with value counts.
- **custom_ac_li**  List. custom_ac converted to list. Used to update crash_actype individual Make with 'Custom'.
- **df3**           DataFrame. df merged with updated crash_actype to get standardized names. Added InjuryScore column.
- **total_score**   DataFrame. Aircraft type and score totaled from df3.




## Data Clenaing and Preparation
***

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('zippedData/AviationData.csv', encoding='latin-1')

<ipython-input-2-46a298358933>:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('zippedData/AviationData.csv', encoding='latin-1')


In [3]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [4]:
df.shape

(88889, 31)

In [5]:
# Drop rows before 2012 so we can match data by year in bts_df
date_mask = (df['Event.Date'] > '2011-12-31')
# THIS RETURNS ALL ROWS GREATER THAN THE DATE PROVIDED ABOVE
df = df.loc[date_mask]

In [6]:
# Rows without an aircraft model type don't help use. Dropping these rows.
df = df.dropna(subset=['Model'])

In [7]:
# Some values in Make are upper, and some are mixed case. Change the Make column to all uppercase
df['Make'] = df['Make'].str.upper()

In [8]:
# Convert Event.Date from string to datetime. Used to filter df on years
df['Event.Date'] = df['Event.Date'].astype('datetime64[ns]')

In [9]:
# Load in T100 data from csv files
files_li = glob.glob('zippedData/T_T100D*.2.csv') 
bts_df = pd.DataFrame(pd.read_csv(files_li[0])) 
for i in range(1,len(files_li)): 
    data = pd.read_csv(files_li[i]) 
    df1 = pd.DataFrame(data) 
    bts_df = pd.concat([df1,bts_df],axis=0, ignore_index=True) 

In [10]:
bts_df.columns

Index(['DISTANCE', 'AIR_TIME', 'UNIQUE_CARRIER_NAME', 'ORIGIN', 'DEST',
       'AIRCRAFT_TYPE', 'YEAR', 'MONTH', 'DISTANCE_GROUP'],
      dtype='object')

In [11]:
bts_df['YEAR'].value_counts()

YEAR
2022    411461
2019    395552
2021    384081
2016    382397
2018    380238
2017    373658
2015    370162
2014    358366
2012    347452
2013    343132
2020    297833
Name: count, dtype: int64

In [12]:
bts_df['UNIQUE_CARRIER_NAME'].value_counts().head(20)

UNIQUE_CARRIER_NAME
Southwest Airlines Co.                  381471
Delta Air Lines Inc.                    348303
United Air Lines Inc.                   301166
SkyWest Airlines Inc.                   237574
American Airlines Inc.                  188528
Federal Express Corporation             186737
ExpressJet Airlines LLC d/b/a aha!      129799
Hageland Aviation Service               122776
Allegiant Air                           122148
United Parcel Service                   107885
Republic Airline                        104241
Alaska Airlines Inc.                     98684
Envoy Air                                92405
Frontier Airlines Inc.                   85449
Endeavor Air Inc.                        85005
Spirit Air Lines                         79363
Grant Aviation                           72125
JetBlue Airways                          69685
PSA Airlines Inc.                        68853
Ryan Air f/k/a Arctic Transportation     65132
Name: count, dtype: int64

## Because the Aviation Crash Data and BTS use different model names for Aircraft type we need to standardize on a set of names.
***

In [14]:
# BTS uses number code to identify aircraft type in it's flight data. The legend is in a seperate csv file. 
# Load in the legend for aircraft type. 
ac_df = pd.read_csv('zippedData/L_AIRCRAFT_TYPE.csv')

In [15]:
# Merge flight data with aircfaft type. This will match BTS type names to the fligt data.
joined_df = pd.merge(bts_df,ac_df,left_on='AIRCRAFT_TYPE',right_on='Code')

In [16]:
# Making a list of aircraft from the flight data. We use this list to assign standard names.
bts_actype = joined_df[['Description']].copy()

In [17]:
# Drop duplicates from flight aircraft 
bts_actype.drop_duplicates(inplace=True)

In [18]:
# Write this to Excel so we can standardize aircraft names.
# bts_actype.to_excel('zippedData/bts_actype2.xlsx')

In [19]:
# Read in our standardized label spreadsheet after edit
bts_map_df = pd.read_excel('zippedData/bts_actype.2.xlsx')

In [20]:
# Convert standardized name spreadsheet to mapping dictionary
bts_map_dic = dict(zip(bts_map_df.Description, bts_map_df.Model))

In [21]:
# Map original aircraft names to standardized name and put in new column
joined_df['NewModel'] = joined_df['Description'].map(bts_map_dic)

In [22]:
joined_df.columns

Index(['DISTANCE', 'AIR_TIME', 'UNIQUE_CARRIER_NAME', 'ORIGIN', 'DEST',
       'AIRCRAFT_TYPE', 'YEAR', 'MONTH', 'DISTANCE_GROUP', 'Code',
       'Description', 'NewModel'],
      dtype='object')

In [23]:
# Convert some intergers to strings
joined_df['NewModel'] = joined_df['NewModel'].astype(str)

In [24]:
# Making a list of aircraft from the NTSB crash data. 
crash_actype = df[['Make', 'Model']].copy()

In [25]:
# We really want to do this on crash_actype
#df['Model'].replace('-', '', inplace=True)
#df['Model'].replace('/', '', inplace=True)
crash_actype['Model'].replace('-', '', inplace=True, regex=True)
crash_actype['Model'].replace('/', '', inplace=True, regex=True)

In [26]:
# Drop duplicates from the crash_actype df.
crash_actype.drop_duplicates(inplace=True)


In [27]:
# Get value counts so we can drop single instance aircraft.
custom_ac = crash_actype['Model'].value_counts().to_frame()

In [28]:
# Convert custom_ac to list. Used to update crash_actype individual Make with 'Custom'.
custom_ac = custom_ac[custom_ac['count'] > 2]
custom_ac_li = custom_ac.index.to_list()


In [29]:
# Lots of duplicates after replacing Make with 'Custom'. Drop duplicates from the crash_actype df
crash_actype.drop_duplicates(inplace=True)

In [30]:
# Replace individual Maker with 'Custom'
for model in custom_ac_li:
    crash_actype.loc[crash_actype['Model'] == model, 'Make'] = 'Custom'

In [31]:
crash_actype['Model'].value_counts()

Model
RV6A             23
SONEX            20
CHALLENGER II    19
RV7              16
SEAREY           15
                 ..
200D              1
PA28R  201T       1
EMB 135KL         1
OH 13HM74A        1
M8 EAGLE          1
Name: count, Length: 4072, dtype: int64

In [32]:
''' 
This function will 
1) take in a model string from crash_actype 
2) match it with a newmodel from joined_df
3) update crash_actype with new_model column
TODO: Replace with faster version.
'''
def match_types(model):
    for newmodel in joined_df['NewModel']:
        if model.startswith(newmodel):
            #print(f'Processing: {model} \t:{newmodel}')
            return newmodel
        

In [33]:
"""
    Creating the NewModel colum takes a lot of CPU and time. It's been saved as a pickle file so we can just
    read it in and save time.
"""
#crash_actype['NewModel'] = [match_types(model) for model in crash_actype['Model']]

In [34]:
# Save df to pickle file.
#crash_actype.to_pickle('zippedData/crash_actype.2.pkl')

In [551]:
# Read in pickle file
crash_actype = pd.read_pickle(r'zippedData/crash_actype.pkl')

In [35]:
crash_actype.shape

(5506, 3)

## Data Modeling
***

In [826]:
# List the top 40 flown aircraft types
joined_df['NewModel'].value_counts().head(40)

NewModel
737         888665
A320        642230
208         219309
RJ200       195787
757         178044
ERJ175      173038
RJ700       169427
MD80        156614
145         153205
206         139183
CRJ900      137718
767         104339
A300         65703
PA31         63511
EMB170       57466
747          44479
1900         44466
DASH8        39522
190          38706
PC12         36956
212          32537
MD11         28799
DC10         27205
DHC2         26880
140          24471
DC9          22351
nan          20616
777          19567
PA32         16926
135          15880
King Air     15591
GA8          12387
402          12274
727          12106
GIV          11323
340B         11120
A330         10326
C185         10051
FALCON        8001
EMB120        7929
Name: count, dtype: int64

In [827]:
# Merge Aircraft Data with crash_actype to get standardized Aircraft Types
df3 = pd.merge(df, crash_actype,left_on='Model',right_on='Model', suffixes=("_l", "_r"))

In [830]:
top_40 = joined_df['NewModel'].value_counts().head(40).index.to_list()

In [831]:
df3 = df3.loc[df3['NewModel'].isin(top_40)]

In [832]:
df3.shape

(2357, 33)

In [833]:
# Some rows have NaN for values. Replace these with 0 value. 
df3['Total.Fatal.Injuries'] = df3['Total.Fatal.Injuries'].fillna(0)
df3['Total.Serious.Injuries'] = df3['Total.Serious.Injuries'].fillna(0)
df3['Total.Minor.Injuries'] = df3['Total.Minor.Injuries'].fillna(0)

In [834]:
# Check replacement
df3['Total.Fatal.Injuries'].isna().sum()


0

In [836]:
# Our scoring system will use 3 points for fatality; 1 point for serious injury and .5 points for minor injury.
df3['Injury_Score'] = df3['Total.Fatal.Injuries'].map(lambda x: x*3)
df3['Injury_Score'] = df3['Injury_Score'] + df3['Total.Serious.Injuries']
df3['Injury_Score'] = df3['Injury_Score'] + (df3['Total.Minor.Injuries'] * .5)

In [839]:
df3.loc[:,['Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries','Injury_Score']].head(5)


,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Injury_Score
2071,0.0,0.0,0.0,0.0
2072,0.0,0.0,0.0,0.0
2073,0.0,0.0,0.0,0.0
2074,0.0,0.0,0.0,0.0
2075,0.0,0.0,0.0,0.0
2076,0.0,13.0,0.0,13.0
2077,127.0,0.0,0.0,381.0
2078,0.0,0.0,0.0,0.0
2079,0.0,0.0,0.0,0.0
2080,0.0,0.0,0.0,0.0


In [840]:
total_score = df3.groupby('NewModel')['Injury_Score'].sum().to_frame()

In [841]:
type(total_score)

pandas.core.frame.DataFrame

In [848]:
total_score['Count'] = joined_df.groupby('NewModel').size()

In [849]:
joined_df.groupby('NewModel').size()


NewModel
105            71
12             88
124           601
135         15880
140         24471
            ...  
S76           166
SIDDELEY      216
TBM850        302
TURBINE      1442
nan         20616
Length: 122, dtype: int64

In [850]:
total_score.columns

Index(['Injury_Score', 'Count', 'Rating'], dtype='object')

In [856]:
total_score['Rating'] = (total_score['Injury_Score'] / total_score['Count']) * 1000

In [857]:
total_score.sort_values('Rating')

,Injury_Score,Count,Rating
NewModel,,,
A300,0.0,65703,0.000000
EMB120,0.0,7929,0.000000
DC9,0.0,22351,0.000000
145,0.0,153205,0.000000
ERJ175,0.5,173038,0.002890
MD11,1.0,28799,0.034723
757,8.0,178044,0.044933
MD80,14.0,156614,0.089392
190,4.0,38706,0.103343


In [911]:
df['Broad.phase.of.flight'].notna().sum()

0